In [ ]:
# Setup library
## install -r requirements.txt
from __future__ import absolute_import, division, print_function, unicode_literals
import os
from operator import itemgetter
import collections

import matplotlib.pylab as plt
# %matplotlib widget
%matplotlib inline

import pandas as pd
import numpy as np
import tensorflow as tf
tf.random.set_seed(99)

In [ ]:
## Load data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
IMG_SHAPE = train_images[0].shape
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
print(f'train_images shape: {train_images.shape}')
print(f'test_images shape: {test_images.shape}')

In [ ]:
## get result labels
fig1 = plt.figure(figsize=(10, 10))
for i in range(25):
    ax = fig1.add_subplot(5, 5, i+1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)
    ax.imshow(test_images[i])
    ax.set_xlabel(class_names[test_labels[i][0]], color='brown')

In [ ]:
metrics = [
    tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy'),
]


def make_model(metrics=metrics, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=IMG_SHAPE))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(len(class_names)))
    
    model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=metrics)
    
    return model


model = make_model()
model.summary()

In [ ]:
class CollectBatchStats(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_losses = []
        self.batch_val_losses = []
        self.batch_acc = []
        self.batch_val_acc = []
    
    def on_epoch_end(self, epoch, logs=None):
        self.batch_losses.append(logs['loss'])
        self.batch_acc.append(logs['accuracy'])
        self.batch_val_losses.append(logs['val_loss'])
        self.batch_val_acc.append(logs['val_accuracy'])
        self.model.reset_metrics()

In [ ]:
epochs = 100
batch_size = 32
steps_per_epoch = np.ceil(len(train_images)/batch_size)
stats = CollectBatchStats()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                  verbose=1,
                                                  patience=10,
                                                  mode='auto',
                                                  restore_best_weights=True)

In [ ]:
history = model.fit(train_images, train_labels,
                    epochs=epochs,
                    steps_per_epoch=steps_per_epoch,
                    batch_size=batch_size,
                    validation_data=(test_images, test_labels),
                    callbacks=[stats, early_stopping])

In [ ]:
# Draw learning curves chart
acc = stats.batch_acc
val_acc = stats.batch_val_acc
loss = stats.batch_losses
val_loss = stats.batch_val_losses

fig2 = plt.figure(figsize=(8, 8))
ax1 = fig2.add_subplot(2, 1, 1)
ax1.plot(acc, label='Training Accuracy')
ax1.plot(val_acc, label='Validation Accuracy')
ax1.legend(loc='lower right')
ax1.set_ylabel('Accuracy')
ax1.set_ylim([0, 1])
ax1.set_title('Training and Validation Accuracy')

ax2 = fig2.add_subplot(2, 1, 2)
ax2.plot(loss, label='Training Loss')
ax2.plot(val_loss, label='Validation Loss')
ax2.legend(loc='upper right')
ax2.set_ylabel('Cross Entropy')
ax2.set_ylim([0,max(ax2.get_ylim())])
ax2.set_title('Training and Validation Loss')
ax2.set_xlabel('epoch')

In [ ]:
# import pickle
# with open('asis.pickle', 'wb') as f:
#     pickle.dump((acc, val_acc, loss, val_loss), f)
print(max(val_acc))

In [ ]:
predicted_batch = model.predict(test_images)
predicted_id = np.argmax(predicted_batch, axis=-1)

label_id = test_labels.reshape(-1,)

con_mat = tf.math.confusion_matrix(label_id, predicted_id)

result_df = pd.DataFrame(con_mat.numpy(), index=class_names, columns=class_names, dtype=int)

print('-- Validation result (Row: Actual Class, Column: Predicted Class) --')
display(result_df)

true_positive = np.diag(result_df)

true_negative = list()
for i in result_df.index:
    t = result_df.drop(i, axis=1)
    t = t.drop(i, axis=0)
    true_negative.append(np.sum(np.sum(t)))
true_negative = np.asarray(true_negative)

false_positive = np.sum(result_df, axis=1) - true_positive

false_negative = np.sum(result_df, axis=0) - true_positive

In [ ]:
accuracy = np.sum(true_positive) / np.sum(np.sum(result_df))
print(f'Accuracy:  {accuracy:>1.10f}')
recall = true_positive / (true_positive + false_negative)
recall[np.isnan(recall)] = 0
recall = np.average(recall)
print(f'Recall:    {recall:>1.10f}')
precision = true_positive / (true_positive + false_positive)
precision[np.isnan(precision)] = 0
precision = np.average(precision)
print(f'Precision: {precision:>1.10f}')
f1_score = 2*((precision*recall)/(precision+recall))
print(f'F1Score:   {f1_score:>1.10f}')